In [28]:
!pip uninstall torch torchvision torchaudio -y

Found existing installation: torch 2.6.0.dev20241112+cu121
Uninstalling torch-2.6.0.dev20241112+cu121:
  Successfully uninstalled torch-2.6.0.dev20241112+cu121
Found existing installation: torchvision 0.20.0.dev20241112+cu121
Uninstalling torchvision-0.20.0.dev20241112+cu121:
  Successfully uninstalled torchvision-0.20.0.dev20241112+cu121
Found existing installation: torchaudio 2.5.0.dev20241112+cu121
Uninstalling torchaudio-2.5.0.dev20241112+cu121:
  Successfully uninstalled torchaudio-2.5.0.dev20241112+cu121


In [30]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu121

Looking in indexes: https://download.pytorch.org/whl/nightly/cu121
  Using cached https://download.pytorch.org/whl/nightly/cu121/torch-2.6.0.dev20241112%2Bcu121-cp311-cp311-linux_x86_64.whl (768.0 MB)
  Using cached https://download.pytorch.org/whl/nightly/cu121/torchvision-0.20.0.dev20241112%2Bcu121-cp311-cp311-linux_x86_64.whl (7.4 MB)
  Using cached https://download.pytorch.org/whl/nightly/cu121/torchaudio-2.5.0.dev20241112%2Bcu121-cp311-cp311-linux_x86_64.whl (3.4 MB)


In [1]:
!python -c "import torch; print(torch.__file__)"
import torch
print(torch.__file__)
print(torch.cuda.is_available())
!python -c "import torch; print(torch.cuda.is_available())"

/mnt/home/douliyang/mlir-workspace/iree-tutorial/myenv/lib/python3.11/site-packages/torch/__init__.py
/mnt/home/douliyang/mlir-workspace/iree-tutorial/myenv/lib/python3.11/site-packages/torch/__init__.py
True
True


In [10]:
torch.manual_seed(0)

class LinearModule(torch.nn.Module):
  def __init__(self, in_features, out_features):
    super().__init__()
    self.weight = torch.nn.Parameter(torch.randn(in_features, out_features))
    self.bias = torch.nn.Parameter(torch.randn(out_features))

  def forward(self, input):
    return (input @ self.weight) + self.bias

linear_module = LinearModule(4, 3)

In [15]:
import torch
print(torch._dynamo.list_backends())

['cudagraphs', 'inductor', 'onnxrt', 'openxla', 'turbine_cpu', 'tvm']


In [16]:
opt_linear_module = torch.compile(linear_module, backend="cudagraphs")
print("Compiled module using Turbine. New module type is", type(opt_linear_module))

Compiled module using Turbine. New module type is <class 'torch._dynamo.eval_frame.OptimizedModule'>


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [19]:
opt_linear_module.to(device)

OptimizedModule(
  (_orig_mod): LinearModule()
)

In [20]:
args = torch.randn(4).to(device)
turbine_output = opt_linear_module(args)

print("Weight:", linear_module.weight)
print("Bias:", linear_module.bias)
print("Args:", args)
print("Output:", turbine_output)

Weight: Parameter containing:
tensor([[ 1.5410, -0.2934, -2.1788],
        [ 0.5684, -1.0845, -1.3986],
        [ 0.4033,  0.8380, -0.7193],
        [-0.4033, -0.5966,  0.1820]], device='cuda:0', requires_grad=True)
Bias: Parameter containing:
tensor([-0.8567,  1.1006, -1.0712], device='cuda:0', requires_grad=True)
Args: tensor([ 0.1227, -0.5663,  0.3731, -0.8920], device='cuda:0')
Output: tensor([-0.4792,  2.5237, -0.9772], device='cuda:0',
       grad_fn=<CompiledFunctionBackward>)
